In [1]:
%pip install fastai
%pip install torch torchvision torchaudio


  Using cached networkx-3.3-py3-none-any.whl.metadata (5.1 kB)
  Using cached joblib-1.4.2-py3-none-any.whl.metadata (5.4 kB)
  Using cached threadpoolctl-3.5.0-py3-none-any.whl.metadata (13 kB)
  Using cached mpmath-1.3.0-py3-none-any.whl.metadata (8.6 kB)
   ---------------------------------------- 0.0/12.2 MB ? eta -:--:--
   ---- ----------------------------------- 1.3/12.2 MB 6.1 MB/s eta 0:00:02
   ------- -------------------------------- 2.4/12.2 MB 6.1 MB/s eta 0:00:02
   --------------- ------------------------ 4.7/12.2 MB 7.3 MB/s eta 0:00:02
   ------------------------ --------------- 7.6/12.2 MB 9.0 MB/s eta 0:00:01
   ----------------------------------- ---- 10.7/12.2 MB 10.3 MB/s eta 0:00:01
   ---------------------------------------  12.1/12.2 MB 10.6 MB/s eta 0:00:01
   ---------------------------------------  12.1/12.2 MB 10.6 MB/s eta 0:00:01
   ---------------------------------------  12.1/12.2 MB 10.6 MB/s eta 0:00:01
   ---------------------------------------- 12.2

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-intel 2.17.0 requires numpy<2.0.0,>=1.23.5; python_version <= "3.11", but you have numpy 2.0.2 which is incompatible.


   ---------------------------------------- 0.0/2.4 MB ? eta -:--:--
   ---------------------------------------- 0.0/2.4 MB ? eta -:--:--
   ---- ----------------------------------- 0.3/2.4 MB ? eta -:--:--
   ------------- -------------------------- 0.8/2.4 MB 1.8 MB/s eta 0:00:01
   --------------------- ------------------ 1.3/2.4 MB 2.1 MB/s eta 0:00:01
   ---------------------------------- ----- 2.1/2.4 MB 2.6 MB/s eta 0:00:01
   ---------------------------------------- 2.4/2.4 MB 2.5 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [4]:
from fastai.vision.all import *
import pathlib

# Stel het pad in naar je afbeeldingen
data_dir = pathlib.Path("../Liesa")

# Controleer of het pad correct is
assert data_dir.exists(), f"Pad {data_dir} bestaat niet."

# DataBlock voor het verwerken van afbeeldingen
dblock = DataBlock(
    blocks=(ImageBlock, CategoryBlock),  # We classificeren afbeeldingen
    get_items=get_image_files,           # Alle afbeeldingsbestanden ophalen
    splitter=RandomSplitter(valid_pct=0.2, seed=123),  # 20% voor validatie
    get_y=parent_label,                  # Labels worden gebaseerd op mappenamen
    item_tfms=Resize(180),               # Afbeeldingen worden geschaald naar 180x180
    batch_tfms=aug_transforms(size=180)  # Basisaugmentatie en schaling
)

# Maak een DataLoaders object
dls = dblock.dataloaders(data_dir, bs=32)

# Controleer de inhoud van de DataLoaders (bijvoorbeeld labels)
print(dls.vocab)  # Om te zien welke categorieën hij heeft herkend

# Het model trainen met een vooraf getrainde ResNet-architectuur
learn = cnn_learner(dls, resnet18, metrics=accuracy)

# Samenvatting van het model
learn.model

# Optionele callback voor vroegtijdig stoppen
early_stopping_cb = EarlyStoppingCallback(monitor='valid_loss', patience=8)

# Train het model
learn.fit_one_cycle(20, cbs=[early_stopping_cb])

# Bewaar het getrainde model als een .pkl bestand
learn.export('ad-model.pkl')


['d', 'q', 'z']


epoch,train_loss,valid_loss,accuracy,time
0,1.798175,1.265335,0.398773,01:26
1,1.764190,1.234777,0.423313,01:27
2,1.670717,1.131547,0.521472,01:25
3,1.641817,1.203317,0.466258,01:39
4,1.581460,1.344212,0.423313,01:46
5,1.519135,1.155218,0.447853,02:29
6,1.473226,1.168255,0.460123,02:07
7,1.383769,1.068607,0.515337,01:28
8,1.334029,1.067417,0.490798,01:34
9,1.267887,1.185776,0.447853,01:14


KeyboardInterrupt: 

In [6]:
%pip install opencv-python
%pip install pyautogui pydirectinput opencv-python Pillow



Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.2.1 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


  Obtaining dependency information for pydirectinput from https://files.pythonhosted.org/packages/c9/8e/f909defe798270afc2241d3a8986d51459e70c37f02d2ba549627b781b2c/PyDirectInput-1.0.4-py3-none-any.whl.metadata
  Using cached PyDirectInput-1.0.4-py3-none-any.whl.metadata (3.9 kB)
Using cached PyDirectInput-1.0.4-py3-none-any.whl (8.9 kB)
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.2.1 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
from fastai.vision.all import *
import pathlib
import numpy as np
import cv2
from PIL import ImageGrab, Image
import pydirectinput
import time
import pathlib
import fastai
import torch

# Fix for path issues between different systems (Linux -> Windows)
temp = pathlib.PosixPath
pathlib.PosixPath = pathlib.WindowsPath

# Laad het FastAI model
learn = load_learner('ad-model3.pkl')

# Restore PosixPath to its original state (if necessary)
pathlib.PosixPath = temp

# Laad het FastAI model
#learn = load_learner('ad-model.pkl')

# Stel de afbeeldingsgrootte in zoals vereist door het model
img_height = 180
img_width = 180

# Haal de verschillende klassen op
data_dir = pathlib.Path("../Screenshots")  # Stel het pad in naar de mappen met data
dls = learn.dls
class_names = dls.vocab
print(class_names)

# Functie voor autonoom rijden
def ad():
    # screen_width = 1920  # Pas dit aan naar je werkelijke schermresolutie
    # screen_height = 1080  # Pas dit aan naar je werkelijke schermresolutie
    # left_half_bbox = (0, 0, screen_width // 2, screen_height)  # Alleen de linker helft van het scherm

    while(True):
        # Maak een screenshot van de linker helft van het scherm
        # printscreen = np.array(ImageGrab.grab(bbox=left_half_bbox))  
        printscreen = np.array(ImageGrab.grab())  

        # Bewerken van afbeelding en voorspelling maken
        processed_img = cv2.cvtColor(printscreen, cv2.COLOR_RGB2GRAY)  # Converteer naar grijswaarden
        processed_img = cv2.Canny(processed_img, threshold1=200, threshold2=300)  # Gebruik Canny edge detection
        cv2.imshow('window', processed_img)  # Toon de verwerkte afbeelding
        processed_img = cv2.cvtColor(processed_img, cv2.COLOR_GRAY2RGB)  # Converteer terug naar RGB voor modelinvoer
        processed_img = Image.fromarray(processed_img)  # Converteer naar PIL afbeelding
        processed_img = processed_img.resize((img_width, img_height))  # Schaal naar vereiste afbeeldingsgrootte

        # Maak voorspelling met FastAI model
        prediction, _, probs = learn.predict(processed_img)
        direction = str(prediction)  # Haal de voorspelde richting op
        print(f'Predicted direction: {direction}')

        # Toetsen indrukken afhankelijk van de voorspelling (op basis van QWERTY)
        if direction == 'd':  # Rechts
            pydirectinput.keyDown('d')  # 'd' blijft hetzelfde voor rechts (QWERTY of AZERTY)
            pydirectinput.keyDown('w')  # Vooruit is 'w' in QWERTY (hoewel het 'z' is op AZERTY)
            time.sleep(0.05)  # Pas de duur aan voor gewenste snelheid
            pydirectinput.keyUp('d')  # Toets loslaten
            pydirectinput.keyUp('w')
        elif direction == 'q':  # Links
            pydirectinput.keyDown('a')  # Links is 'a' in QWERTY (hoewel het 'q' is op AZERTY)
            pydirectinput.keyDown('w')  # Vooruit met 'w' in QWERTY
            time.sleep(0.05)  # Pas de duur aan voor gewenste snelheid
            pydirectinput.keyUp('a')  # Toets loslaten
            pydirectinput.keyUp('w')
        elif direction == 'z':  # Rechtdoor
            pydirectinput.keyDown('w')  # Vooruit is 'w' in QWERTY (hoewel het 'z' is op AZERTY)
            time.sleep(0.05)
            pydirectinput.keyUp('w')  # Toets loslaten


       


        # Stop als de 'q'-toets wordt ingedrukt
        if cv2.waitKey(25) & 0xFF == ord('q'):
            cv2.destroyAllWindows()
            break

# Start autonoom rijden
ad()


['d', 'q', 'z']


Predicted direction: z


Predicted direction: d


Predicted direction: d


Predicted direction: z


...

Predicted direction: z


Predicted direction: z


Predicted direction: z


Predicted direction: z


KeyboardInterrupt: 